# Insight Project --Birding Big Year--

In this project I intend to determine a way to see all the birds one can see on a single state, for a given time window.  For all those birdirers that want to get to the top 100 of their state on ebrid, this will be the perfect tool. The user will input the state, home address (or lat,lon), time window and birds that already have been seen*. This last one (*) is an optional thing.

In [1]:
import numpy as np
from datetime import datetime
import geopandas as gpd
import os
import struct
import pickle
import googlemaps
import pandas as pd
import psycopg2

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable

from sklearn.cluster import DBSCAN

import SetCover
import DBScaner
import Definition

def save_fig(name):
    fig.savefig(name,dpi=80,bbox_inches='tight', pad_inches=0.02, format = 'png')

%matplotlib inline

# The ebird Data

I will start with a singe state. Since the ebird API limits the type of request I can make, I have a downloaded the cvs file.  I'm using the last two full years of data but in reality the alorithm should be train with more data and just tested on the last year.

In [2]:
# import importlib
# importlib.reload(DBScaner)

In [3]:
dbname = Definition.DBNAME
username = Definition.USERNAME
pswd = Definition.PSWD

In [7]:
## Now try the same queries, but in python!

# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

# query:
sql_query = """
SELECT * FROM data_lower WHERE year = 2018;
"""
dfTrain = pd.read_sql_query(sql_query,con)

DatabaseError: Execution failed on sql '
SELECT * FROM wisconsin_birds WHERE year = 2018;
': relation "wisconsin_birds" does not exist
LINE 2: SELECT * FROM wisconsin_birds WHERE year = 2018;
                      ^


In [5]:
dfTrain.head()

,index,sampling_event_identifier,common name,locality,latitude,longitude,observation_date,all_species_reported,year_week,year_day,year
0,3,S46395227,Red-eyed Vireo,Riverside Park (Urban Ecology Center),43.067967,-87.892686,2018-06-07,1,23,158,2018
1,4,S47723667,Pine Siskin,Schlitz Audubon Nature Center,43.175688,-87.890421,2018-08-08,1,32,220,2018
2,5,S44649385,Mallard,Manitowoc Lakefront,44.092794,-87.650170,2018-04-16,1,16,106,2018
3,15,S43252514,Mallard,UW Lakeshore Nature Preserve--Willow Creek Woo...,43.077869,-89.421544,2018-02-27,1,9,58,2018
4,16,S46958065,Mallard,County V Ponds (Dane Co.),43.247843,-89.429111,2018-07-03,1,27,184,2018


In [ ]:
# dfAll = pd.read_csv('./ebd_US-WY_201801_201912_relApr-2020/ebd_US-WY_201801_201912_relApr-2020.txt'
#                 ,delimiter="\t")

dfAll = pd.read_csv('../ebd_US-WI_201801_201912_relApr-2020/ebd_US-WI_201801_201912_relApr-2020.txt'
                ,delimiter="\t", usecols=['CATEGORY', 'LOCALITY TYPE', 'ALL SPECIES REPORTED', 'APPROVED',
                                         'SAMPLING EVENT IDENTIFIER', 'COMMON NAME', 'LOCALITY', 
                                          'LATITUDE', 'LONGITUDE',
                                          'OBSERVATION DATE', 'ALL SPECIES REPORTED'])

# dfAll = pd.read_csv('./ebd_US-WI_201001_201812_relApr-2020/ebd_US-WI_201001_201812_relApr-2020.txt'
#                 ,delimiter="\t")

I add sertain condition to satify completnes fo the data, public locations and only bird species (i.e. no hybirds). `dfReduce` will contian all the information I will be using.

In [ ]:
dfAll = dfAll[(dfAll['CATEGORY'] == 'species') & (dfAll['LOCALITY TYPE'] == 'H')
              & (dfAll['ALL SPECIES REPORTED'] == 1)  & (dfAll['APPROVED'] == 1)]

In [ ]:
dfReduce = dfAll.filter(['SAMPLING EVENT IDENTIFIER', 'COMMON NAME', 'LOCALITY',
              'LATITUDE', 'LONGITUDE', 'OBSERVATION DATE', 'ALL SPECIES REPORTED']) 
dfReduce['OBSERVATION DATE'] = pd.to_datetime(dfReduce['OBSERVATION DATE'])
dfReduce['YEAR WEEK'] = dfReduce['OBSERVATION DATE'].dt.strftime('%W')
dfReduce['YEAR DAY'] = dfReduce['OBSERVATION DATE'].dt.strftime('%j')
dfReduce['YEAR'] = dfReduce['OBSERVATION DATE'].dt.strftime('%Y')
dfReduce['YEAR WEEK'] = pd.to_numeric(dfReduce['YEAR WEEK'])

In [ ]:
dfReduce.head(5)

dfReduce contains both my train set and my validation set.  In this case I will use the last year as my validation set (2019) and all the previous information as my train set.

In [ ]:
dfValidation = dfReduce[dfReduce['YEAR']==2019]

In [ ]:
dfTrain = dfReduce[dfReduce['YEAR']!=2019]

In [ ]:
del dfReduce
del dfAll

In [ ]:
len(np.unique(dfTrain['LOCALITY'].values))

# Clustering using BDSCAN

BDSCAN is a density clustering that will tell where is popular for people to go birding (based on the desnity of hotsopts).  I will define a cluster as having atleast 3 point and with a maximum distance of 0.05degrees or about 5km.  With that I will optain where does each hotspot ('LOCALITY') belongs to. If '-1' they are not part of any cluster.

In [ ]:
dfcluster, labels, n_clusters_ = DBScaner.cluster_selection(dfTrain)

In [ ]:
dfProb = DBScaner.cluster_center(dfcluster, dfTrain)

### Now some good plots

In [ ]:
country = gpd.read_file('/Users/casanova/DocumentsHere/Insight/Project/gz_2010_us_040_00_5m.json')

In [ ]:
# state = 'Wyoming'
state = 'Wisconsin'

unique_labels = set(labels)
colors = [plt.cm.viridis_r(each)
          for each in np.linspace(0, 1, len(unique_labels))]

In [ ]:
fig, ax = plt.subplots(1, figsize=(7,8))
base = country[country['NAME'].isin([state]) == True].plot(ax=ax, color='#3B3C6E', alpha = 0.3)
for i, clust in enumerate(labels):
    ax.scatter(np.array(dfcluster['LONGITUDE'])[i],np.array(dfcluster['LATITUDE'])[i], color = colors[clust])
# ax.scatter(dfcluster['LONGITUDE'],dfcluster['LATITUDE'])
ax.scatter(dfProb['LONGITUDE'],dfProb['LATITUDE'], marker = 'x', color = 'r', s=80)
ax.set_ylabel(r'Latitude [$^o$]')
ax.set_xlabel(r'Longitude [$^o$]')
# save_fig('/Users/casanova/DocumentsHere/Insight/{}-static.png'.format(state))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, figsize=(7,8))
base = country[country['NAME'].isin([state]) == True].plot(ax=ax, color='#3B3C6E', alpha = 0.3)
ax.scatter(dfcluster['LONGITUDE'],dfcluster['LATITUDE'])#, color = colors[clust])
# ax.scatter(dfProb['LONGITUDE'],dfProb['LATITUDE'], marker = 'x', color = 'r', s=80)
ax.set_ylabel(r'Latitude [$^o$]')
ax.set_xlabel(r'Longitude [$^o$]')

plt.show()
# save_fig('/Users/casanova/DocumentsHere/Insight/WI.png')

#### Now the bird probability.

`dfbdscan` have the information of where each of the hotspots lay, in terms of their cluster.  Now in order to constuct a path is important to mask the probabilites of the of seeing a particular bird with T or F on a weekly basis.  This is critical in order to construc the sets.

In [ ]:
dfProb = dfTrain.merge(dfbdscan.filter(['LOCALITY','BD CLUSTER']),
                            left_on='LOCALITY', right_on='LOCALITY', how = 'left').filter(['COMMON NAME','ALL SPECIES REPORTED','YEAR WEEK', 'BD CLUSTER'])

In [ ]:
dfProb.head(5)

In [ ]:
nTime = 54
nLoc = n_clusters_
setMat = np.empty((nTime,nLoc), dtype=object)

In [ ]:
for week in range(0,nTime):
    dfProbA = dfProb[dfProb['YEAR WEEK']== week]
    dfProb1 = dfProbA.groupby(['COMMON NAME','BD CLUSTER']).sum().filter(['ALL SPECIES REPORTED']).reset_index()
    dfProb1.rename(columns = {'ALL SPECIES REPORTED':'POS OBS'}, inplace=True)
    dfProb2 = dfProbA.groupby(['BD CLUSTER']).sum().filter(['ALL SPECIES REPORTED']).reset_index()
    dfProb2.rename(columns = {'ALL SPECIES REPORTED':'TOT OBS'}, inplace=True)
    dfProb3 = dfProb1.merge(dfProb2, left_on='BD CLUSTER', right_on='BD CLUSTER', how = 'left')
    dfProb3['POS PROB'] = dfProb3['POS OBS']/dfProb3['TOT OBS']
    for loc in range(0,nLoc):
        dfWeek = dfProb3[dfProb3['BD CLUSTER'] == loc]
        dfWeek['TF aa'] = list(map(lambda x: 0 if x < 0.02 else 1, dfWeek['POS PROB']))
        setMat[week,loc] = set(dfWeek[dfWeek['TF aa'] == 1]['COMMON NAME'].values)
        

In [ ]:
pickle.dump(setMat, open("./2dSetLocations.p", "wb" ))

In [ ]:
ToMakeUniverse = list(setMat.flatten())
Universe = set(e for s in ToMakeUniverse for e in s)

In [ ]:
# list(Universe)

# Here we go!!!!!

First user inputs some coordinates.
Then the coordinates get translated to a cluster.
That give us the first set (first week)
Then we obtain the rest of the sets. The key here is to back track a set to an actual 'x,t' entry so we can have a route.
Display in some way that list of locations!  (Probabily using the centroid maps or coordinates).

In [ ]:
# userInputLat,userInputLon = 44, -110
userInputLat,userInputLon = 43.069511, -89.396723

userInput = [userInputLat,userInputLon]
print(userInput)

On the first week I most see:

In [ ]:
setMatLoaded = pickle.load(open("./2dSetLocations.p", "rb" ))

The hole list of bird that we are planing to see are:

In [ ]:
ToMakeUniverse = list(setMatLoaded.flatten())
Universe = set(e for s in ToMakeUniverse for e in s)

In [ ]:
print('With a total of', len(list(Universe)), 'birds')

In [ ]:
setList, locList = SetCover.set_cover_greedy(Universe, ToMakeUniverse)


In [ ]:
locList = np.sort(locList)

In [ ]:
nTime, nLoc = setMatLoaded.shape
locMat = np.linspace(1,nTime*nLoc,nTime*nLoc).reshape(nTime,nLoc)

In [ ]:
outList = []
for element in locList:
    a,b = np.where(locMat == element)
    outList.append('On week {}, you need to be at location {}'.format(a[0],b[0]))
    

In [ ]:
outList

# Plaing with google distances

In [ ]:
gmaps = googlemaps.Client(key='')

In [ ]:
distMat = np.empty((nTime,n_clusters_))
countBreak = divmod(n_clusters_,100)

for i in range(0,countBreak[0]):
    distanceMatGmaps = gmaps.distance_matrix(origins = (userInput[0],userInput[1]), 
                                         destinations=list(coorHotspot[100*i:100*(i+1)]),
                                         mode = 'driving', units = 'metric')
    for j in range(100*i,100*(i+1)):
        jj = j - 100*i
        try:
            distMat[:,j] = distanceMatGmaps['rows'][0]['elements'][jj]['duration']['value']/3600
        except KeyError:
            distMat[:,j] = 100
        
distanceMatGmaps = gmaps.distance_matrix(origins = (userInput[0],userInput[1]), 
                                         destinations=list(coorHotspot[countBreak[0]*100:]),
                                         mode = 'driving', units = 'metric')

for j in range(countBreak[0]*100,countBreak[0]*100+countBreak[1]):
    jj = j - countBreak[0]*100
    try:
        distMat[:,j] = distanceMatGmaps['rows'][0]['elements'][jj]['duration']['value']/3600
    except KeyError:
        distMat[:,j] = 100

In [ ]:
distMat

In [ ]:
a, b = SetCover.set_cover_weighted_greedy(Universe, ToMakeUniverse,list(distMat.flatten()))

In [ ]:
locListb = np.sort(b)

In [ ]:
locListb

In [ ]:
outList = []
for element in locListb:
    a,b = np.where(locMat == element)
    outList.append('On week {}, you need to be at location {}'.format(a[0],b[0]))

In [ ]:
outList